# Leitura da Base

In [3]:
import pandas as pd
import re
import os
import joblib
import collections
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy
import nltk
from rouge_score import rouge_scorer
from collections import Counter

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Larissa
[nltk_data]     Fossato\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
if os.path.exists('lista_papers'):
    lista_papers = joblib.load('lista_papers')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/Documents_xml')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r')
        lines.append(f.readlines())
        f.close()

    lista_papers=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_papers.append(texto)

        joblib.dump(lista_papers, 'lista_papers')

In [4]:
len(lista_papers)

1009

In [5]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
papers_com_indice = dict(zip(indices, lista_papers))

# Estudo da Base

In [6]:
import string

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

In [9]:
def get_indices_preprocessamento(x, funcao_preprocessamento):
    """ alterar a funcao_preprocessamento entre os parâmetros:
    preprocessa_intro, preprocessa_abstract, preprocessa_conclusion"""
    papers_processados=[]
    indices=[]
    for i,j in zip(list(x.keys()),list(x.values())):
        try:
            papers_processados.append(funcao_preprocessamento(j))
            indices.append(i)
        except:
            None
    return indices

In [10]:
indices_papers_com_abstract   = get_indices_preprocessamento(papers_com_indice, preprocessa_abstract)
indices_papers_com_introducao = get_indices_preprocessamento(papers_com_indice, preprocessa_intro)
indices_papers_com_conclusao  = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion)

In [11]:
print('qtd de papers com abstract', len(indices_papers_com_abstract))
print('qtd de papers com introdução', len(indices_papers_com_introducao))
print('qtd de papers com conclusão', len(indices_papers_com_conclusao))

print('---------------------------------')

print('qtd total de papers', len(lista_papers))

qtd de papers com abstract 924
qtd de papers com introdução 802
qtd de papers com conclusão 711
---------------------------------
qtd total de papers 1009


Contagem das sessões a partir do título faltante para melhoria do Regex

In [12]:
papers_sem_abstract = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_abstract)
papers_sem_introducao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_introducao)
papers_sem_conclusao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_conclusao)

In [13]:
def contador_sessoes(x):
    """input: dicionário de papers
       output: lista contendo o título das sessões"""
    titulos_sessoes=[]
    for i in x.values():
        regIter = re.finditer('(<SECTION(.*?)>)', i, re.DOTALL|re.MULTILINE)
        textos = [ t.group(2) for t in regIter]
        titulos_sessoes.append(textos)

    x=[]
    for i in titulos_sessoes:
        for j in i:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern,str(j))
            pattern2 = r'[0-9]'
            new_string = re.sub(pattern2, '', matches[0])
            x.append(new_string.strip())

    frequency = collections.Counter(x)
    return sorted(frequency.items(), key=lambda x:x[1], reverse=True)

In [14]:
contador_sessoes(papers_sem_abstract)

[('.', 20),
 ('Introduction', 12),
 ('Acknowledgements', 6),
 ('Results', 5),
 ('', 4),
 ('Related Work', 4),
 ('Acknowledgments', 3),
 ('Abstract  Introduction', 3),
 ('Data and Evaluation', 3),
 ('Concluding Remarks', 3),
 ('Conclusion', 2),
 ('Experimental Results', 2),
 ('Conclusions and Future Work', 2),
 ('Conditional Random Fields', 2),
 ('The Joyce System in the Ulysses User Interface', 1),
 ('The Structure of Joyce', 1),
 ('The Text Planner', 1),
 ('The Sentence Planner', 1),
 ('The Linguistic Realizer', 1),
 ('An Example', 1),
 ('ENTER AGENT #&lt;information&gt; OBJECT INCOMPONENT Black Box&gt; LOCATION #&lt;PORT P&gt;',
  1),
 ('Porting the System', 1),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('Input Structure', 1),
 ('System Architecture', 1),
 ('Linguistic Knowledge Bases', 1),
 ('Interfaces', 1),
 ('System Performance', 1),
 ('Status', 1),
 ('. Corpus-based Methods', 1),
 ('Arabic Lang

In [15]:
contador_sessoes(papers_sem_introducao)

[('.', 549),
 ('', 83),
 ('Acknowledgments', 63),
 ('Conclusion', 15),
 ('Motivation', 13),
 ('Acknowledgements', 9),
 ('Conclusions', 9),
 ('Related Work', 8),
 ('Definition', 8),
 ('Evaluation', 7),
 ('Background', 5),
 ('Acknowledgment', 4),
 ('Experiments', 4),
 ('Results', 4),
 ('Appendix', 3),
 ('References', 3),
 ('Conclusion and Future Work', 3),
 ('Discussion', 3),
 ('ACKNOWLEDGEMENTS', 3),
 ('Conclusions and future work', 2),
 ('Overview', 2),
 ('Related Work.', 2),
 ('. Results', 2),
 ('AT&amp;T Bell Laboratories AT&amp;T Bell Laboratories', 2),
 ('Applications', 2),
 ('Approach', 2),
 ('Conclusions and Future Work', 2),
 ('Summary', 2),
 ('English', 2),
 ('Resources', 2),
 ('Discussion and related work', 1),
 ('(-. &amp;quot;NN&amp;quot;)', 1),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('.. The Amount of Training Data Required', 1),
 ('Proper Name Identification in Natural Language Process

In [16]:
def preprocessa_conclusion1(corpus: str):
    regIter = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

In [41]:
def preprocessa_conclusion4(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1  = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*Acknowledgments.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2  = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*Acknowledgements.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    texto=textos1
    for i in textos2:
        if i not in texto:
            texto.append(i)
    for i in (textos3)&(textos2):
        if i not in texto:
             texto.append(i)
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

In [18]:
def preprocessa_conclusion2(corpus: str):
    regIter = re.finditer('(<SECTION.*Acknowledgments.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

In [19]:
def preprocessa_conclusion4(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1  = [ t.group(2) for t in regIter]
    regIter2 = re.finditer('(<SECTION.*Acknowledgments.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2  = [ t.group(2) for t in regIter]
    regIter3 = re.finditer('(<SECTION.*Acknowledgements.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter]
    textos=textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in (textos3)&(textos2):
        if i not in textos:
             textos.append(i)
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

In [38]:
def preprocessa_conclusion4(corpus: str):
    regIter = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    textos=textos1
    return textos

In [45]:
def preprocessa_conclusion4(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1  = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*Acknowledgments.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2  = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*Acknowledgements.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    texto=textos1
    for i in textos2:
        if i not in texto:
            texto.append(i)
    for i in (textos3)&(textos2):
        if i not in texto:
             texto.append(i)
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return textos1

In [84]:
def preprocessa_conclusion4(corpus: str):
    regIter = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    return textos

In [103]:
def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

In [109]:
teste  = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion)

In [142]:
def preprocessa_conclusion4(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1  = [ t.group(2) for t in regIter1]
    #regIter2 = re.finditer('(<SECTION.*Acknowledgments.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    #textos2  = [ t.group(2) for t in regIter2]
    #regIter3 = re.finditer('(<SECTION.*Acknowledgements.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    #textos3 = [ t.group(2) for t in regIter3]
    #texto=textos1
    for i in textos2:
        if i not in texto:
            texto.append(i)
    for i in (textos3)&(textos2):
        if i not in texto:
             texto.append(i)
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return textos1

In [143]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [183]:
def preprocessa_conclusion4(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*Acknowledgments.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*Acknowledgements.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    return textos3

In [193]:
def preprocessa_conclusion4(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*Acknowledgments.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*Acknowledgements.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
    return txt

In [194]:
x=[]
for i in lista_papers:
    try:
        x.append(preprocessa_conclusion4(i))
    except:
        None
x=remove_values_from_list(x,[])

In [195]:
len(x)

827

In [117]:
len(preprocessa_conclusion(lista_papers[0]))

2123

In [110]:
len(teste)

711

In [77]:
teste

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 15,
 16,
 18,
 19,
 20,
 22,
 24,
 31,
 32,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 48,
 50,
 51,
 53,
 54,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 74,
 90,
 97,
 99,
 100,
 101,
 102,
 103,
 104,
 106,
 108,
 110,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 134,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 166,
 167,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 178,
 179,
 180,
 181,
 182,
 184,
 185,
 186,
 187,
 188,
 192,
 198,
 199,
 200,
 202,
 203,
 204,
 205,
 206,
 208,
 209,
 210,
 222,
 225,
 226,
 231,
 233,
 234,
 237,
 238,
 241,
 242,
 243,
 247,
 248,
 250,
 253,
 255,
 256,
 258,
 260,
 265,
 267,
 271,
 272,
 275,
 276,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 289,
 290,
 296,
 299,
 302,
 305,
 307,
 3

In [ ]:
def preprocessa_conclusion3(corpus: str):
    regIter = re.finditer('(<SECTION.*Acknowledgements.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        #txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('&amp;quot;', '', txt)
        lista.append(txt)
    return txt

In [ ]:
indices_papers_com_conclusao   = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion)
indices_papers_com_conclusao2  = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion2)
indices_papers_com_conclusao3  = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion3)

x=[]
for i in indices_papers_com_conclusao2:
    if i not in (indices_papers_com_conclusao+indices_papers_com_conclusao3):
        x.append(i)
indices_papers_com_conclusao2 = x 

x=[]
for i in indices_papers_com_conclusao3:
    if i not in indices_papers_com_conclusao:
        x.append(i)
indices_papers_com_conclusao3 = x 

conclusao1 =  dict((key,value) for key, value in papers_com_indice.items() if key in  indices_papers_com_conclusao)
conclusao2 =  dict((key,value) for key, value in papers_com_indice.items() if key in  indices_papers_com_conclusao2)
conclusao3 = dict((key,value) for key, value in papers_com_indice.items() if key in  indices_papers_com_conclusao3)

conclusao = dict(conclusao1)
conclusao.update(conclusao2)
conclusao.update(conclusao3)

In [ ]:
len(d4)

In [ ]:
len(set(d4.keys()))

In [ ]:
#Results, Discussion, Concluding remarks, Experiments and Results, Conclustion and Future Research
contador_sessoes(papers_sem_conclusao)

Identificação dos papers com as 3 sessões simultâneamente

In [ ]:
indice_papers=[]
for idx in list(indices_papers_com_abstract):
    if idx in list(indices_papers_com_introducao):
        if idx in list(indices_papers_com_conclusao):
            indice_papers.append(idx)

In [ ]:
print('Quantidade de papers com as três sessões simulataneamente:',len(indice_papers))

In [ ]:
indices_faltantes=[]
for i in indices:
    if i not in indice_papers:
        indices_faltantes.append(i)

In [ ]:
print('Quantidade de papers sem as três sessões:',len(indices_faltantes))

papers_de_fora = dict((key,value) for key, value in papers_com_indice.items() if key  in indices_faltantes)

contador_sessoes(papers_de_fora)

In [ ]:
papers = dict((key,value) for key, value in papers_com_indice.items() if key in indice_papers)

Seguir o estudo com os artigos armazenados na variável `papers`. \
Os armazenados na variável `papers_de_fora` não entrarão no estudo.

# Separação das Seções

In [ ]:
#Definição do Abstract
abstract=[]
for i in papers.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

In [ ]:
token_abstract

# Experimento 1

## Word 2 Vec

In [ ]:
if os.path.exists('model'):
    model = joblib.load('model')
else:
    model = Word2Vec(tokens, min_count=1, epochs=100)
    joblib.dump(model,'model')

In [ ]:
len(model.wv.key_to_index)

In [ ]:
string=re.sub("\(.*?\)","",string)

In [ ]:
model.wv.most_similar('no', topn=10)

In [ ]:
model.wv.most_similar('contrary', topn=10)

In [ ]:
model.wv.most_similar('statistical', topn=10)

## GloVe

https://stackoverflow.com/questions/64261546/how-to-solve-error-microsoft-visual-c-14-0-or-greater-is-required-when-inst

from glove import Corpus, Glove

#Creating a corpus object
corpus = Corpus() 

#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus.fit(lines, window=10)

glove = Glove(no_components=5, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

https://github.com/maciejkula/glove-python

## Fast Text

import fasttext


model = fasttext.train_unsupervised('data.txt', model='skipgram')


model = fasttext.train_unsupervised('data.txt', model='cbow')

https://pypi.org/project/fasttext/

## Medida de similaridade

In [ ]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = txt.strip()
        lista.append(txt)
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = txt.strip()
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = txt.strip()
        lista.append(txt)
    return txt

In [ ]:
def funcao_frase_palavra(frases_abstract):
    """input: lista de frases
        output: lista de palavras de cada frase"""
    palavras_frases=[]
    for i in frases_abstract:
        palavras_frases.append(i.split(' '))
    try:
        palavras_frases.remove('')
    except:
        return palavras_frases
    return palavras_frases

def vetor_frases(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno
    
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [ ]:
#Definição do Abstract
abstract=[]
for i in papers.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

In [ ]:
#Introduction
frases_intro=[]
for i in intro:
    frases_intro.append(remove_values_from_list(i.split('.'),''))
    
    
frases_intro3=[]
for i in frases_intro:
    frases_intro2=[]
    for j in i:
        frases_intro2.append(j.strip())
    frases_intro3.append(frases_intro2)
    
frases_intro = frases_intro3 


palavras_intro=[]
for i in frases_intro:
    palavras_intro.append(funcao_frase_palavra(i))
    
vetor_intro=[]
for i in palavras_intro:
    vetor_intro.append(vetor_frases(i))

In [ ]:
#Abstract
frases_abstract=[]
for i in abstract:
    frases_abstract.append(remove_values_from_list(i.split('.'),''))
    
    
frases_abstract3=[]
for i in frases_abstract:
    frases_abstract2=[]
    for j in i:
        frases_abstract2.append(j.strip())
    frases_abstract3.append(frases_abstract2)
    
frases_abstract = frases_abstract3 


palavras_abstract=[]
for i in frases_abstract:
    palavras_abstract.append(funcao_frase_palavra(i))
    
vetor_abstract=[]
for i in palavras_abstract:
    vetor_abstract.append(vetor_frases(i))

In [ ]:
#Conclusion
frases_conclusion=[]
for i in conclusion:
    frases_conclusion.append(remove_values_from_list(i.split('.'),''))
    
    
frases_conclusion3=[]
for i in frases_conclusion:
    frases_conclusion2=[]
    for j in i:
        frases_conclusion2.append(j.strip())
    frases_conclusion3.append(frases_conclusion2)
frases_conclusion = frases_conclusion3 

palavras_conclusion=[]
for i in frases_conclusion:
    palavras_conclusion.append(funcao_frase_palavra(i))
    
vetor_conclusion=[]
for i in palavras_conclusion:
    vetor_conclusion.append(vetor_frases(i))

In [ ]:
dicionario_similaridade(vetor_abstract[0], vetor_intro[0])

In [ ]:
#Similaridade entre Abstract e Introdução
x = []
for i,j in zip(range(len(vetor_abstract)),range(len(vetor_intro))):
    x.append(dicionario_similaridade(vetor_abstract[i], vetor_intro[j]))

In [ ]:
top_3_indices_ab_intro = []
for i in range(len(x)):
    top_3_indices_ab_intro.append(sorted(x[i], key=x[i].get, reverse=True)[:3])

In [ ]:
top_3_indices_ab_intro[3]

In [ ]:
for i in top_3_indices_ab_intro[0]:
    print(x[0][i])

In [ ]:
lista_1=[]
for i in top_3_indices_ab_intro:
    top3=[]
    for j in i:
        top3.append(j[0])
    lista_1.append(list(set(top3)))

In [ ]:
lista_1[0]

In [ ]:
#Similaridade entre Abstract e Conclusão
x = []
for i,j in zip(range(len(vetor_abstract)),range(len(vetor_conclusion))):
    x.append(dicionario_similaridade(vetor_abstract[i], vetor_conclusion[j]))

In [ ]:
#maiores indices
top_3_indices_ab_conc = []
for i in range(len(x)):
    top_3_indices_ab_conc.append(sorted(x[i], key=x[i].get, reverse=True)[:3])

In [ ]:
top_3_indices_ab_conc[0]

In [ ]:
for i in top_3_indices_ab_conc[0]:
    print(x[0][i])

In [ ]:
lista_2=[]
for i in top_3_indices_ab_conc:
    top3=[]
    for j in i:
        top3.append(j[0])
    lista_2.append(list(set(top3)))

In [ ]:
lista_2[1]

In [ ]:
lista_1[1]

In [ ]:
lista_2[0:5]

Intersecção entre as frases do abstract que são similares simultâneamente à introdução e a conclusão

In [ ]:
d = [[1,2,3,4], [2,3,4], [3,4,5,6,7]]
set.intersection(*map(set,d))

In [ ]:
indices_objetivo=[]
for i,j in zip(lista_1,lista_2):
    indices_objetivo.append(list(set.intersection(*map(set,[i,j]))))

#Pegando os índices em comum entre as seções abstract+intro e abstract+conc
indices_objetivo=[]
for i, j in zip(lista_1,lista_2):
    idx_um_art=[]
    for h, z in zip(i,j):
        if h==z:
            idx_um_art.append(h)
    indices_objetivo.append(idx_um_art)
lista_2

#Pegando os índices em comum entre as seções abstract+intro e abstract+conc
indices_objetivo=[]
for i, j in zip(lista_1,lista_2):
    idx_um_art=[]
    for h, z in zip(i,j):
        if h==z:
            idx_um_art.append(h)
    indices_objetivo.append(idx_um_art)

In [ ]:
frases_abstract[3]

In [ ]:
indices_objetivo[0:5]

`indices_objetivo` índices da frase objetivo

In [ ]:
frases_abstract[0][1]

In [ ]:
indices_objetivo[0]

In [ ]:
frases_abstract[0][1]

In [ ]:
frases_abstract[0][3]

In [ ]:
papers_total=[]
for idx_f,idx_ind in zip(range(len(frases_abstract[0:3])),indices_objetivo[0:3]):
    lista_one=[]
    for j in idx_ind:
        lista_one.append(frases_abstract[idx_f][j])
        #print(idx_f)
        #print(j)
        #print(frases_abstract[idx_f][j])
    papers_total.append(lista_one)

In [ ]:
papers_total

In [ ]:
frase_objetivo=[]
for idx_f,idx_ind in zip(range(len(frases_abstract)),indices_objetivo):
    lista_one=[]
    for j in idx_ind:
        lista_one.append(frases_abstract[idx_f][j])
        #print(idx_f)
        #print(j)
        #print(frases_abstract[idx_f][j])
    frase_objetivo.append(lista_one)

In [ ]:
frase_objetivo

frase_objetivo = []
for i in range(len(frases_abstract)):
    if i in indices_objetivo:
        frase_objetivo.append(frases_abstract[i])

frase_objetivo

Saída para treinamento do modelo de linguagem:
`frase_objetivo`

# Leitura da Sumarização Padrão Ouro

In [ ]:
if os.path.exists('lista_padrao_ouro'):
    lista_padrao_ouro = joblib.load('lista_padrao_ouro')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'summary/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/summary')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r', encoding='utf-8' )
        lines.append(f.readlines())
        f.close()

    lista_padrao_ouro=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_papers.append(texto)

        joblib.dump(lista_padrao_ouro, 'lista_padrao_ouro')

In [ ]:
len(lista_padrao_ouro)

In [ ]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
padrao_ouro = dict(zip(indices, lista_padrao_ouro))

Filtro dos sumários correspondentes aos papers que possuem as Seções Abstract, Introdução e Conclusão

In [ ]:
selecao_padrao_ouro = dict((key,value) for key, value in padrao_ouro.items() if key in  indice_papers)

In [ ]:
len(selecao_padrao_ouro)

# ROUGE SCORE
Aplicação do `ROUGE` entre as frase identificadas como objetivo e as frases padrão ouro

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def pre_processamento_ouro(txt):
    txt = re.sub(r'\n.*','', txt)
    return txt

In [ ]:
selecao_padrao_ouro = list(selecao_padrao_ouro.values())

In [ ]:
lista_frases_padrao_ouro=[]
for i in selecao_padrao_ouro:
    lista_frases_padrao_ouro.append(i.split('.'))

for i,j in zip(frase_objetivo[0:2],lista_frases_padrao_ouro[0:2]):
    for h in range(len(i)):
        for z in range(len(j)):
            print(scorer.score(i[h],j[z])['rouge1'].fmeasure)

In [ ]:
rouge_total=[]
for i,j in zip(frase_objetivo,lista_frases_padrao_ouro):
    rouge_1={}
    for h in range(len(i)):
        for z in range(len(j)):
            rouge_1[(h,z)] = (scorer.score(i[h],j[z])['rouge1'].fmeasure)
    rouge_total.append(rouge_1)

In [ ]:
lista_frases_padrao_ouro

#conferindo
rouge_1={}
for i in range(len(frase_objetivo[0])):
    for j in range(len(frases_padra_ouro)):
        rouge_1[(i,j)] = (scorer.score(frase_objetivo[0][i],frases_padra_ouro[j])['rouge1'].fmeasure)
rouge_1

In [ ]:
def get_max(dic, coord, val):
    return max(filter(lambda item: item[0][coord] == val, dic.items()), key=lambda x: x[1])

In [ ]:
rouge_total

In [ ]:
maximo_rouge_por_paper =[]
for h in rouge_total:
    x=[]
    for i in set([i[0] for i in h]):
        x.append(get_max(h,0,i))
    maximo_rouge_por_paper.append(x)
maximo_rouge_por_paper

x=[]
for i in maximo_rouge_por_paper:
    soma=0
    for j in i:
        soma = soma + j[1]
    #print(len(i))
    try:
        x.append(soma)
    except:
        x.append(0)
x

In [ ]:
precisao=[]
for i in maximo_rouge_por_paper:
    soma=0
    for j in i:
        soma = soma + j[1]
    #print(len(i))
    try:
        precisao.append(soma/len(i))
    except:
        precisao.append(0)

In [ ]:
precisao

In [ ]:
for i in lista_frases_padrao_ouro:
    print(len(i))

In [ ]:
1/7

In [ ]:
revocacao=[]
for i,h in zip(maximo_rouge_por_paper,lista_frases_padrao_ouro):
    soma=0
    for j in i:
        soma = soma + j[1]
    #print(len(i))
    try:
        revocacao.append(soma/len(h))
    except:
        revocacao.append(0)
revocacao

frases_padra_ouro ta errado acima

In [ ]:
len(lista_frases_padrao_ouro[0])

In [ ]:
Counter(precisao)

In [ ]:
sorted(Counter(precisao), reverse=True)

In [ ]:
Counter(precisao)

In [ ]:
Counter(revocacao)

In [ ]:
sorted(Counter(revocacao), reverse=True)

In [ ]:
len(precisao)

In [ ]:
max(revocacao)

tentar guardar a identação - precisão e recall de quais papers?\
testar a remoção das stop words do word2vec\
adicionar os termos similares na conclusão\
glove e fasttext\
treino do model language\
--------
correções na quali

# Modelo de Linguagem